In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr

import matplotlib.pyplot as plt

from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error

from spicy_snow.retrieval import retrieval_from_parameters

# import functions for downloading
from spicy_snow.download.sentinel1 import s1_img_search, hyp3_pipeline, download_hyp3, combine_s1_images
from spicy_snow.download.forest_cover import download_fcf
from spicy_snow.download.snow_cover import download_snow_cover

# import functions for pre-processing
from spicy_snow.processing.s1_preprocessing import merge_partial_s1_images, s1_orbit_averaging,\
s1_clip_outliers, subset_s1_images, ims_water_mask, s1_incidence_angle_masking, merge_s1_subsets

# import the functions for snow_index calculation
from spicy_snow.processing.snow_index import calc_delta_VV, calc_delta_cross_ratio, \
    calc_delta_gamma, clip_delta_gamma_outlier, calc_snow_index, calc_snow_index_to_snow_depth

# import the functions for wet snow flag
from spicy_snow.processing.wet_snow import id_newly_frozen_snow, id_newly_wet_snow, \
    id_wet_negative_si, flag_wet_snow

# setup root logger
from spicy_snow.utils.spicy_logging import setup_logging

# fischer z test
from causallearn.utils.cit import CIT

In [2]:
in_dir = Path('~/scratch/spicy/SnowEx-Data/').expanduser().resolve()
data_dir = Path('~/scratch/spicy/SnowEx-Data/').expanduser().resolve()
dss = {fp.stem: xr.open_dataset(fp) for fp in in_dir.glob('*.nc')}

# Create parameter space
A = np.round(np.arange(1, 3.1, 0.5), 2)
B = np.round(np.arange(0, 2.01, 0.25), 2)
C = np.round(np.arange(0.25, 1.001, 0.25), 2)

In [3]:
def s1_to_sd(ds, A = 1.5, B = 0.1, C = 0.6):
    # mask out outliers in incidence angle
    ds = s1_incidence_angle_masking(ds)
    
    # subset dataset by flight_dir and platform
    dict_ds = subset_s1_images(ds)

    for subset_name, subset_ds in dict_ds.items():
        # average each orbit to overall mean
        dict_ds[subset_name] = s1_orbit_averaging(subset_ds)
        # clip outlier values of backscatter to overall mean
        dict_ds[subset_name] = s1_clip_outliers(subset_ds)
    
    # recombine subsets
    ds = merge_s1_subsets(dict_ds)

    # calculate confidence interval
    # ds = add_confidence_angle(ds)

    ## Snow Index Steps
    # log.info("Calculating snow index")
    # calculate delta CR and delta VV
    ds = calc_delta_cross_ratio(ds, A = A)
    ds = calc_delta_VV(ds)

    # calculate delta gamma with delta CR and delta VV with FCF
    ds = calc_delta_gamma(ds, B = B)

    # clip outliers of delta gamma
    ds = clip_delta_gamma_outlier(ds)

    # calculate snow_index from delta_gamma
    ds = calc_snow_index(ds, ims_masking = True)

    # convert snow index to snow depth
    ds = calc_snow_index_to_snow_depth(ds, C = C)

    return ds

import warnings
def get_gaussian_stats(da):
    arr = da.values
    warnings.filterwarnings("ignore", message="Mean of empty slice")
    warnings.filterwarnings("ignore", message="Degrees of freedom <= 0 ")
    return arr.size, np.nanmean(arr), np.nanstd(arr), arr.shape

In [4]:
def bias(x, y): return np.mean(x - y)

def get_stats(x, y, nrmse = False):
    if type(x) == xr.DataArray: x = x.values.ravel()
    if type(y) == xr.DataArray: y = y.values.ravel()
    if type(x) == list: x = np.array(x)
    if type(y) == list: y = np.array(y)
    idx = (~np.isnan(x)) & (~np.isnan(y))
    x, y = x[idx], y[idx]
    r, p = pearsonr(x, y)
    b = bias(x, y)
    mae = mean_absolute_error(x, y)
    rmse = mean_squared_error(x, y, squared = False)

    if nrmse:
        nrmse_value = rmse / np.mean(x)
        return r, b, mae, rmse, nrmse_value

    return r, b, mae, rmse

from scipy.stats import norm
def fischerz(truth, x1, x2):
    idx = (~np.isnan(truth)) & (~np.isnan(x1)) & (~np.isnan(x2))
    truth, x1, x2 = truth[idx], x1[idx], x2[idx]
    n = len(x1)
    cor1 = pearsonr(truth, x1).statistic
    cor2 = pearsonr(truth, x2).statistic
    fischer1 = 0.5*np.log((1+cor1)/(1-cor1))
    fischer2 = 0.5*np.log((1+cor2)/(1-cor2))
    expected_sd = np.sqrt(1/(n-3))
    return 2 * (1 - norm(0, expected_sd).cdf(np.abs(fischer1 - fischer2)))

# incidence angel random field with parameterization

In [11]:
for stem, full_ds in dss.items():

    if stem == 'Frasier_2020-02-11':
        im_date = pd.to_datetime('2020-02-16')
    else:
        im_date = pd.to_datetime(full_ds.sel(time = full_ds.attrs['lidar-flight-time'], method = 'nearest').time.values.ravel()[0])

    d_days = im_date - pd.to_datetime(full_ds.attrs['lidar-flight-time'])
    site_name = stem.replace('_', ' ').replace('Frasier', 'Fraser').split('-')[0]
    # if site_name == 'Mores 2020' or site_name == 'Fraser 2021' or site_name == 'Dry Creek 2020' or site_name == 'Banner 2021' or site_name == 'Little Cottonwood 2021' or\
    #     site_name == 'Mores 2021' or site_name == 'Banner 2020':
    #     continue

    # create random dataset of VV and VH
    r_ds = full_ds.copy(deep = True)
    for ts in r_ds.time:
        inc = full_ds.sel(time = ts, band = 'inc')['s1']
        for band in ['VV', 'VH']:
            for inc_low, inc_high in [[0, 10], [10, 20], [20, 30], [30, 40], [40, 50], [50, 60], [60, 70]]:
                inc_low, inc_high = np.deg2rad(inc_low), np.deg2rad(inc_high)
                n, u, std, shape = get_gaussian_stats(full_ds['s1'].sel(band = band, time = ts).where((inc > inc_low) & (inc < inc_high)))
                rand = np.random.normal(size = full_ds['lidar-sd'].size, loc = u, scale = std).reshape(full_ds['lidar-sd'].shape)
                r_ds['s1'].loc[dict(time = ts, band = band)] = r_ds['s1'].loc[dict(time = ts, band = band)].where((inc > inc_low) & (inc < inc_high), rand)
    
    # holds rmse and r for optimizing rand dataset
    rand_rmse_ds = xr.DataArray(np.zeros([len(A), len(B), len(C)]), dims = ['A', 'B', 'C'], coords = [A, B, C])
    rand_r_ds = xr.DataArray(np.zeros([len(A), len(B), len(C)]), dims = ['A', 'B', 'C'], coords = [A, B, C])

    optimize random dataset
    for a in A:
        for b in B:
            for c in C:
                r_ds = s1_to_sd(r_ds, A = a, B = b, C = c)
                rand_r, rand_b, rand_mae, rand_rmse, rand_nrmse = get_stats(r_ds['snow_depth'].sel(time = im_date, method = 'nearest'), r_ds['lidar-sd'], nrmse = True)
                rand_rmse_ds.loc[dict(A = a, B = b, C = C)] = rand_rmse
                rand_r_ds.loc[dict(A = a, B = b, C = C)] = rand_r
    
    # get parameters
    a_best = rand_r_ds.max(['B', 'C']).idxmax('A')
    b_best = rand_r_ds.max(['C', 'A']).idxmax('B')
    c_best = rand_rmse_ds.sel(A = a_best, B = b_best).idxmin('C')
    print(f"A: {a_best.data.ravel()[0]} B: {b_best.data.ravel()[0]} C: {c_best.data.ravel()[0]}")
    # a_best, b_best, c_best = 1, 0.0, 0.25

    r_ds = s1_to_sd(r_ds, A= a_best, B = b_best, C = c_best).sel(time = im_date, method = 'nearest')
    ds = full_ds.sel(time = im_date, method = 'nearest')

    # print values
    print(site_name)

    r, b, mae, rmse, nrmse = get_stats(ds['snow_depth'], ds['lidar-sd'], nrmse = True)
    rand_r, rand_b, rand_mae, rand_rmse, rand_nrmse = get_stats(r_ds['snow_depth'], r_ds['lidar-sd'], nrmse = True)
    fischer_z = fischerz(ds['lidar-sd'].data.ravel(), ds['snow_depth'].data.ravel(), r_ds['snow_depth'].data.ravel())

    print(f'S1: R: {r}. RMSE: {rmse}. RANDOM: R: {rand_r_ds.sel(A = a_best, B = b_best, C = c_best).data.ravel()[0]} RMSE: {rand_rmse_ds.sel(A = a_best, B = b_best, C = c_best).data.ravel()[0]}. FischerZ: {fischer_z}')
    # print(f'S1: R: {r}. RMSE: {rmse}. RANDOM: R: {rand_r} RMSE: {rand_rmse}. FischerZ: {fischer_z}')

Mores 2020
S1: R: 0.08426767662873491. RMSE: 1.0739974121237865. RANDOM: R: 0.1505348795647083 RMSE: 1.4093288033084126. FischerZ: 0.0001242480417045222
Fraser 2021
S1: R: 0.18382002487927696. RMSE: 0.6541369851067282. RANDOM: R: 0.10667524188054645 RMSE: 0.7513895781508385. FischerZ: 0.0005879646394355564
Dry Creek 2020
S1: R: 0.20510866035642356. RMSE: 0.7382410700288716. RANDOM: R: 0.06621667487901962 RMSE: 0.9229266598088253. FischerZ: 0.0
Banner 2021
S1: R: 0.41537871427426587. RMSE: 0.8915869451365991. RANDOM: R: 0.09440825698128871 RMSE: 1.052883594648055. FischerZ: 0.0
Little Cottonwood 2021
S1: R: 0.5397881461260303. RMSE: 1.065947341300313. RANDOM: R: 0.11148637741974951 RMSE: 1.2961281002120792. FischerZ: 0.0
Mores 2021
S1: R: 0.39756464003682557. RMSE: 0.9145271045588742. RANDOM: R: 0.005409224595314001 RMSE: 1.180942715143895. FischerZ: 0.0
Banner 2020
S1: R: 0.40022758224160293. RMSE: 0.9950183007831572. RANDOM: R: 0.23595462256583005 RMSE: 1.2194473555172656. FischerZ: 0

# all sites combined

In [51]:
sd = []
s1 = []
rands = []
for stem, full_ds in dss.items():

    if stem == 'Frasier_2020-02-11':
        im_date = pd.to_datetime('2020-02-16')
    else:
        im_date = pd.to_datetime(full_ds.sel(time = full_ds.attrs['lidar-flight-time'], method = 'nearest').time.values.ravel()[0])

    d_days = im_date - pd.to_datetime(full_ds.attrs['lidar-flight-time'])
    site_name = stem.replace('_', ' ').replace('Frasier', 'Fraser').split('-')[0]
    # if site_name == 'Mores 2020' or site_name == 'Fraser 2021' or site_name == 'Dry Creek 2020' or site_name == 'Banner 2021' or site_name == 'Little Cottonwood 2021' or\
    #     site_name == 'Mores 2021' or site_name == 'Banner 2020':
    #     continue

    # create random dataset of VV and VH
    r_ds = full_ds[['s1','fcf','ims']].copy(deep = True)
    for ts in r_ds.time:
        inc = full_ds.sel(time = ts, band = 'inc')['s1']
        for band in ['VV', 'VH']:
            for inc_low, inc_high in [[0, 10], [10, 20], [20, 30], [30, 40], [40, 50], [50, 60], [60, 70]]:
                inc_low, inc_high = np.deg2rad(inc_low), np.deg2rad(inc_high)
                n, u, std, shape = get_gaussian_stats(full_ds['s1'].sel(band = band, time = ts).where((inc > inc_low) & (inc < inc_high)))
                rand = np.random.normal(size = full_ds['lidar-sd'].size, loc = u, scale = std).reshape(full_ds['lidar-sd'].shape)
                r_ds['s1'].loc[dict(time = ts, band = band)] = r_ds['s1'].loc[dict(time = ts, band = band)].where((inc > inc_low) & (inc < inc_high) & (~np.isnan(full_ds['s1'].sel(time = ts, band = band))), rand)
    
    # holds rmse and r for optimizing rand dataset
    # rand_rmse_ds = xr.DataArray(np.zeros([len(A), len(B), len(C)]), dims = ['A', 'B', 'C'], coords = [A, B, C])
    # rand_r_ds = xr.DataArray(np.zeros([len(A), len(B), len(C)]), dims = ['A', 'B', 'C'], coords = [A, B, C])

    # # optimize random dataset
    # for a in A:
    #     for b in B:
    #         for c in C:
    #             r_ds = s1_to_sd(r_ds, A = a, B = b, C = c)
    #             rand_r, rand_b, rand_mae, rand_rmse, rand_nrmse = get_stats(r_ds['snow_depth'].sel(time = im_date, method = 'nearest'), r_ds['lidar-sd'], nrmse = True)
    #             rand_rmse_ds.loc[dict(A = a, B = b, C = C)] = rand_rmse
    #             rand_r_ds.loc[dict(A = a, B = b, C = C)] = rand_r
    
    # get parameters
    # a_best = rand_r_ds.max(['B', 'C']).idxmax('A')
    # b_best = rand_r_ds.max(['C', 'A']).idxmax('B')
    # c_best = rand_rmse_ds.sel(A = a_best, B = b_best).idxmin('C')
    # print(f"A: {a_best.data.ravel()[0]} B: {b_best.data.ravel()[0]} C: {c_best.data.ravel()[0]}")
    best_a, best_b, best_c = 1, 0.0, 0.25
    r_ds = s1_to_sd(r_ds[['s1','ims','fcf']], A= best_a, B = best_b, C = best_c).sel(time = im_date, method = 'nearest')
    ds = full_ds.sel(time = im_date, method = 'nearest')
    x1, x2 = r_ds['snow_depth'].data.ravel(), ds['lidar-sd'].data.ravel()
    idx = (~np.isnan(x1)) & (~np.isnan(x2))
    print(site_name)
    print(pearsonr(x1[idx], x2[idx]))

    rands.append(r_ds['snow_depth'].data.ravel())
    # print(np.concatenate(rands).size)
    s1.append(ds['snow_depth'].data.ravel())
    sd.append(ds['lidar-sd'].data.ravel())
    x1, x2 = np.concatenate(rands), np.concatenate(sd)
    idx = (~np.isnan(x1)) & (~np.isnan(x2))
    print(np.sum(idx))

    print(pearsonr(x1[idx], x2[idx]))

rands, s1, sd = np.concatenate(rands), np.concatenate(s1), np.concatenate(sd)
r, b, mae, rmse, nrmse = get_stats(s1, sd, nrmse = True)
rand_r, rand_b, rand_mae, rand_rmse, rand_nrmse = get_stats(rands, sd, nrmse = True)
fischer_z = fischerz(sd, s1, rands)

print(f'S1: R: {r}. RMSE: {rmse}. RANDOM: R: {rand_r} RMSE: {rand_rmse}. FischerZ: {fischer_z}')

Mores 2020
PearsonRResult(statistic=0.12213905729392673, pvalue=3.181663272479397e-14)
3836
PearsonRResult(statistic=0.12213905729392673, pvalue=3.181663272479397e-14)
Fraser 2021
PearsonRResult(statistic=0.12349825108791976, pvalue=2.1379287495439696e-25)
10895
PearsonRResult(statistic=0.3959287495208208, pvalue=0.0)


KeyboardInterrupt: 

In [53]:
rands[0].size + rands[1].size

17250

In [54]:
np.concatenate(rands).size

17250

# no incidence angle accounting

In [ ]:
for stem, full_ds in dss.items():
    # fig, axes = plt.subplots(1,4, figsize = (12, 8))
    # im_time = ds.attrs['lidar-flight-time']
    # ds['snow_index'].sel(time = im_time, method = 'nearest').where(~ds['lidar-sd'].isnull()).plot(ax = axes[0])
    # r_ds['snow_index'].sel(time = im_time, method = 'nearest').where(~ds['lidar-sd'].isnull()).plot(ax = axes[1])
    # ds['lidar-sd'].plot(ax = axes[2])
    # ds['lidar-dem'].plot(ax = axes[3])

    if stem == 'Frasier_2020-02-11':
        im_date = pd.to_datetime('2020-02-16')
    else:
        im_date = pd.to_datetime(full_ds.sel(time = full_ds.attrs['lidar-flight-time'], method = 'nearest').time.values.ravel()[0])

    d_days = im_date - pd.to_datetime(full_ds.attrs['lidar-flight-time'])
    site_name = stem.replace('_', ' ').replace('Frasier', 'Fraser').split('-')[0]

    r_ds = full_ds.copy(deep = True)
    for ts in r_ds.time:
        for band in ['VV', 'VH']:
            n, u, std, shape = get_gaussian_stats(full_ds['s1'].sel(band = band, time = ts))
            r_ds['s1'].loc[dict(time = ts, band = band)] = np.random.normal(size = n, loc = u, scale = std).reshape(shape)

    r_ds = s1_to_sd(r_ds)

    ds = full_ds.sel(time = im_date, method = 'nearest')

    print(site_name)
    r, b, mae, rmse, nrmse = get_stats(ds['snow_depth'], ds['lidar-sd'], nrmse = True)
    rand_r, rand_b, rand_mae, rand_rmse, rand_nrmse = get_stats(r_ds['snow_depth'].sel(time = im_date, method = 'nearest'), r_ds['lidar-sd'], nrmse = True)
    print(f'S1: R: {r}. RMSE: {nrmse}. RANDOM: R: {rand_r} RMSE: {rand_nrmse}')